<img src="https://i.imgur.com/6U6q5jQ.png"/>

<a target="_blank" href="https://colab.research.google.com/github/SocialAnalytics-StrategicIntelligence/TableOperations/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Operations on Data Frames


Let me get the data on dengue from [Peru](https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue):

In [1]:
import pandas as pd
linkData="https://github.com/SocialAnalytics-StrategicIntelligence/TableOperations/raw/main/dataFiles/dengue_ok.pkl"
dengue = pd.read_pickle(linkData)
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398943 entries, 0 to 398942
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   departamento  398943 non-null  object        
 1   provincia     398943 non-null  object        
 2   distrito      398943 non-null  object        
 3   ano           398943 non-null  int64         
 4   semana        398943 non-null  int64         
 5   sexo          398943 non-null  object        
 6   edad          398943 non-null  int64         
 7   enfermedad    398943 non-null  category      
 8   year          398931 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(3), object(4)
memory usage: 24.7+ MB


In [2]:
# some exploration
dengue.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,ano,semana,edad,year
count,398943.00000,398943.00000,398943.00000,398931.00000
mean,2015.06170,22.61685,29.97476,.5f
min,2000.00000,1.00000,0.00000,.5f
25%,2011.00000,11.00000,15.00000,.5f
50%,2016.00000,19.00000,27.00000,.5f
75%,2020.00000,34.00000,42.00000,.5f
max,2022.00000,53.00000,106.00000,.5f
std,6.14862,14.89333,18.53260,nan


Each row is a person:

In [3]:
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01


If we wanted to count people, creating a column of ones helps:

In [4]:
dengue=dengue.assign(case=1)
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year,case
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01,1
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01,1
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01,1
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01,1
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01,1


Let's start creating _data from these data_!

# Average Age - by Year and Symptoms

## Aggregation

Having people, we need to count them by some grouping variable, in this case year (_ano_) and dengue status.


In [10]:
indexList=['ano','enfermedad']
aggregator={'edad': ['mean','median'], 'case':['sum']}
ByYear_stats=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYear_stats.head(20)

edad          case
                       mean median    sum
ano  enfermedad                          
2000 SIN_SEÑALES  29.508788   27.0   4324
2001 SIN_SEÑALES  30.634282   28.0  15851
     GRAVE        31.572614   28.0    241
2002 SIN_SEÑALES  26.960178   24.0   6278
     ALARMA       12.000000   12.0      1
     GRAVE        21.928571   19.0     14
2003 SIN_SEÑALES  28.947719   27.0   2850
     GRAVE        38.000000   30.0     15
2004 SIN_SEÑALES  28.863269   26.0   7928
     GRAVE        23.794118   19.0     34
2005 SIN_SEÑALES  26.891200   24.0   5000
     GRAVE        23.533333   23.0     15
2006 SIN_SEÑALES  27.436258   25.0   3624
     GRAVE        13.500000    8.5      4
2007 SIN_SEÑALES  28.676543   26.0   5299
     GRAVE        34.285714   30.0     35
2008 SIN_SEÑALES  28.941008   26.0  10493
     GRAVE        27.342857   22.0     35
2009 SIN_SEÑALES  30.180729   28.0  11271
     GRAVE        17.727273   13.0     11

In [11]:
# notice hierarchy: multindex
ByYear_stats.columns

MultiIndex([('edad',   'mean'),
            ('edad', 'median'),
            ('case',    'sum')],
           )

For easier manipualtion outside Python, we could flatten the index hierarchy:

In [12]:
# ok?
["_".join(name) for name in ByYear_stats.columns]

['edad_mean', 'edad_median', 'case_sum']

In [13]:
# changing
ByYear_stats.columns=["_".join(name) for name in ByYear_stats.columns]
ByYear_stats.head(20)

edad_mean  edad_median  case_sum
ano  enfermedad                                   
2000 SIN_SEÑALES  29.508788         27.0      4324
2001 SIN_SEÑALES  30.634282         28.0     15851
     GRAVE        31.572614         28.0       241
2002 SIN_SEÑALES  26.960178         24.0      6278
     ALARMA       12.000000         12.0         1
     GRAVE        21.928571         19.0        14
2003 SIN_SEÑALES  28.947719         27.0      2850
     GRAVE        38.000000         30.0        15
2004 SIN_SEÑALES  28.863269         26.0      7928
     GRAVE        23.794118         19.0        34
2005 SIN_SEÑALES  26.891200         24.0      5000
     GRAVE        23.533333         23.0        15
2006 SIN_SEÑALES  27.436258         25.0      3624
     GRAVE        13.500000          8.5         4
2007 SIN_SEÑALES  28.676543         26.0      5299
     GRAVE        34.285714         30.0        35
2008 SIN_SEÑALES  28.941008         26.0     10493
     GRAVE        27.342857         22.0        35
2009 SIN_SEÑALES  30.180729         28.0     11271
     GRAVE        17.727273         13.0        11

In [14]:
# final look:
ByYear_stats.reset_index(drop=False,inplace=True)
ByYear_stats.head(20)

,ano,enfermedad,edad_mean,edad_median,case_sum
0,2000,SIN_SEÑALES,29.508788,27.0,4324
1,2001,SIN_SEÑALES,30.634282,28.0,15851
2,2001,GRAVE,31.572614,28.0,241
3,2002,SIN_SEÑALES,26.960178,24.0,6278
4,2002,ALARMA,12.000000,12.0,1
5,2002,GRAVE,21.928571,19.0,14
6,2003,SIN_SEÑALES,28.947719,27.0,2850
7,2003,GRAVE,38.000000,30.0,15
8,2004,SIN_SEÑALES,28.863269,26.0,7928
9,2004,GRAVE,23.794118,19.0,34


Notice a particular data type:

In [15]:
ByYear_stats.enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)

Saving to CSV will erase that _dtype_ attribute. Then, use pickle file format: 

In [16]:
ByYear_stats.to_pickle('dataFiles/ByYear_stats.pkl') # this can be read in R.

## Reshaping

Notice the variables are in three columns: **edad_mean** /	**edad_median** / 	**case_sum**. We could reshape those columns to a long format: 

In [17]:
theVarsAsIndex=['ano','enfermedad']

# stacking  and resetting index
ByYear_LongStats=ByYear_stats.set_index(theVarsAsIndex).stack().reset_index()

#result
ByYear_LongStats

,ano,enfermedad,level_2,0
0,2000,SIN_SEÑALES,edad_mean,29.508788
1,2000,SIN_SEÑALES,edad_median,27.000000
2,2000,SIN_SEÑALES,case_sum,4324.000000
3,2001,SIN_SEÑALES,edad_mean,30.634282
4,2001,SIN_SEÑALES,edad_median,28.000000
...,...,...,...,...
172,2022,ALARMA,edad_median,25.000000
173,2022,ALARMA,case_sum,7370.000000
174,2022,GRAVE,edad_mean,35.146226
175,2022,GRAVE,edad_median,32.000000


In [18]:
# just renaming
ByYear_LongStats.rename(columns={'level_2':'statsName',0:'statsValue'},inplace=True)
ByYear_LongStats

,ano,enfermedad,statsName,statsValue
0,2000,SIN_SEÑALES,edad_mean,29.508788
1,2000,SIN_SEÑALES,edad_median,27.000000
2,2000,SIN_SEÑALES,case_sum,4324.000000
3,2001,SIN_SEÑALES,edad_mean,30.634282
4,2001,SIN_SEÑALES,edad_median,28.000000
...,...,...,...,...
172,2022,ALARMA,edad_median,25.000000
173,2022,ALARMA,case_sum,7370.000000
174,2022,GRAVE,edad_mean,35.146226
175,2022,GRAVE,edad_median,32.000000


In [19]:
# still ordinal?
ByYear_LongStats.enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)

In [20]:
ByYear_LongStats.to_pickle('dataFiles/ByYear_LongStats.pkl')

# Dengue by Location (Province)

## Aggregating

We can redo the previous process, adding _departamento_ and _province_: 

In [21]:
indexList=['ano','departamento','provincia','enfermedad']
aggregator={'case':['sum']}
ByYearPlace=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYearPlace

case
                                             sum
ano  departamento provincia     enfermedad      
2000 AMAZONAS     BAGUA         SIN_SEÑALES  215
                  UTCUBAMBA     SIN_SEÑALES   58
     CAJAMARCA    CUTERVO       SIN_SEÑALES    2
                  JAEN          SIN_SEÑALES   16
     HUANUCO      LEONCIO PRADO SIN_SEÑALES   29
...                                          ...
2022 UCAYALI      PADRE ABAD    SIN_SEÑALES  412
                                ALARMA        87
                                GRAVE          2
                  PURUS         SIN_SEÑALES    1
                                ALARMA         1

[2300 rows x 1 columns]

Before flattening the output data frame in long format, you could create a wide shape:

## Long to wide

In [22]:
#simply
ByYearPlace.unstack()

case             
                                           sum             
enfermedad                         SIN_SEÑALES ALARMA GRAVE
ano  departamento provincia                                
2000 AMAZONAS     BAGUA                  215.0    NaN   NaN
                  UTCUBAMBA               58.0    NaN   NaN
     CAJAMARCA    CUTERVO                  2.0    NaN   NaN
                  JAEN                    16.0    NaN   NaN
     HUANUCO      LEONCIO PRADO           29.0    NaN   NaN
...                                        ...    ...   ...
2022 TUMBES       ZARUMILLA               89.0    5.0   NaN
     UCAYALI      ATALAYA                542.0   92.0   2.0
                  CORONEL PORTILLO      2680.0  499.0  23.0
                  PADRE ABAD             412.0   87.0   2.0
                  PURUS                    1.0    1.0   NaN

[1310 rows x 3 columns]

In [23]:
# a more familiar look
ByYearPlace_wide=ByYearPlace.unstack().reset_index()
ByYearPlace_wide

ano departamento         provincia        case             
                                                        sum             
enfermedad                                      SIN_SEÑALES ALARMA GRAVE
0           2000     AMAZONAS             BAGUA       215.0    NaN   NaN
1           2000     AMAZONAS         UTCUBAMBA        58.0    NaN   NaN
2           2000    CAJAMARCA           CUTERVO         2.0    NaN   NaN
3           2000    CAJAMARCA              JAEN        16.0    NaN   NaN
4           2000      HUANUCO     LEONCIO PRADO        29.0    NaN   NaN
...          ...          ...               ...         ...    ...   ...
1305        2022       TUMBES         ZARUMILLA        89.0    5.0   NaN
1306        2022      UCAYALI           ATALAYA       542.0   92.0   2.0
1307        2022      UCAYALI  CORONEL PORTILLO      2680.0  499.0  23.0
1308        2022      UCAYALI        PADRE ABAD       412.0   87.0   2.0
1309        2022      UCAYALI             PURUS         1.0    1.0   NaN

[1310 rows x 6 columns]

In [24]:
# zero instead of missing
ByYearPlace_wide.fillna(0,inplace=True)
ByYearPlace_wide

ano departamento         provincia        case             
                                                        sum             
enfermedad                                      SIN_SEÑALES ALARMA GRAVE
0           2000     AMAZONAS             BAGUA       215.0    0.0   0.0
1           2000     AMAZONAS         UTCUBAMBA        58.0    0.0   0.0
2           2000    CAJAMARCA           CUTERVO         2.0    0.0   0.0
3           2000    CAJAMARCA              JAEN        16.0    0.0   0.0
4           2000      HUANUCO     LEONCIO PRADO        29.0    0.0   0.0
...          ...          ...               ...         ...    ...   ...
1305        2022       TUMBES         ZARUMILLA        89.0    5.0   0.0
1306        2022      UCAYALI           ATALAYA       542.0   92.0   2.0
1307        2022      UCAYALI  CORONEL PORTILLO      2680.0  499.0  23.0
1308        2022      UCAYALI        PADRE ABAD       412.0   87.0   2.0
1309        2022      UCAYALI             PURUS         1.0    1.0   0.0

[1310 rows x 6 columns]

In [25]:
# you expected
ByYearPlace_wide.columns

MultiIndex([(         'ano',    '',            ''),
            ('departamento',    '',            ''),
            (   'provincia',    '',            ''),
            (        'case', 'sum', 'SIN_SEÑALES'),
            (        'case', 'sum',      'ALARMA'),
            (        'case', 'sum',       'GRAVE')],
           names=[None, None, 'enfermedad'])

In [26]:
#prepare
["_".join(names) if names[1]!='' else names[0] for names in ByYearPlace_wide.columns]

['ano',
 'departamento',
 'provincia',
 'case_sum_SIN_SEÑALES',
 'case_sum_ALARMA',
 'case_sum_GRAVE']

In [27]:
# change
ByYearPlace_wide.columns=["_".join(names) if names[1]!='' else names[0] for names in ByYearPlace_wide.columns]
ByYearPlace_wide

,ano,departamento,provincia,case_sum_SIN_SEÑALES,case_sum_ALARMA,case_sum_GRAVE
0,2000,AMAZONAS,BAGUA,215.0,0.0,0.0
1,2000,AMAZONAS,UTCUBAMBA,58.0,0.0,0.0
2,2000,CAJAMARCA,CUTERVO,2.0,0.0,0.0
3,2000,CAJAMARCA,JAEN,16.0,0.0,0.0
4,2000,HUANUCO,LEONCIO PRADO,29.0,0.0,0.0
...,...,...,...,...,...,...
1305,2022,TUMBES,ZARUMILLA,89.0,5.0,0.0
1306,2022,UCAYALI,ATALAYA,542.0,92.0,2.0
1307,2022,UCAYALI,CORONEL PORTILLO,2680.0,499.0,23.0
1308,2022,UCAYALI,PADRE ABAD,412.0,87.0,2.0


What about finding the _provincia_ most affected in a _departamento_?

In [28]:
where = ByYearPlace_wide.groupby(['ano','departamento'])['case_sum_ALARMA'].idxmax()
worst_prov_year = ByYearPlace_wide.loc[where].reset_index(drop=True)
worst_prov_year

,ano,departamento,provincia,case_sum_SIN_SEÑALES,case_sum_ALARMA,case_sum_GRAVE
0,2000,AMAZONAS,BAGUA,215.0,0.0,0.0
1,2000,CAJAMARCA,CUTERVO,2.0,0.0,0.0
2,2000,HUANUCO,LEONCIO PRADO,29.0,0.0,0.0
3,2000,JUNIN,CHANCHAMAYO,4.0,0.0,0.0
4,2000,LA LIBERTAD,TRUJILLO,894.0,0.0,0.0
...,...,...,...,...,...,...
366,2022,PIURA,PIURA,3471.0,667.0,27.0
367,2022,PUNO,CARABAYA,25.0,0.0,0.0
368,2022,SAN MARTIN,SAN MARTIN,770.0,350.0,6.0
369,2022,TUMBES,TUMBES,515.0,28.0,0.0


In [31]:
# amount of worst provinces per region
len(worst_prov_year.provincia.value_counts())

59

In [32]:
# amount of worst provinces per region - cleaner
len(worst_prov_year[worst_prov_year.case_sum_ALARMA>0].provincia.value_counts())

43

## Filtering

Lets' filter some rows based on what we just computed:

In [33]:
worst_ProvYear_alarma=worst_prov_year[worst_prov_year.case_sum_ALARMA>0].loc[:,['departamento','provincia']]
worst_ProvYear_alarma.reset_index(drop=True,inplace=True)
worst_ProvYear_alarma

,departamento,provincia
0,LORETO,MAYNAS
1,JUNIN,SATIPO
2,LORETO,MAYNAS
3,MADRE DE DIOS,TAMBOPATA
4,PIURA,PIURA
...,...,...
198,PASCO,OXAPAMPA
199,PIURA,PIURA
200,SAN MARTIN,SAN MARTIN
201,TUMBES,TUMBES


In [34]:
# adding a column of ones
worst_ProvYear_alarma['case']=1
worst_ProvYear_alarma

,departamento,provincia,case
0,LORETO,MAYNAS,1
1,JUNIN,SATIPO,1
2,LORETO,MAYNAS,1
3,MADRE DE DIOS,TAMBOPATA,1
4,PIURA,PIURA,1
...,...,...,...
198,PASCO,OXAPAMPA,1
199,PIURA,PIURA,1
200,SAN MARTIN,SAN MARTIN,1
201,TUMBES,TUMBES,1


## Frequency table

With filtered data, let's create a crosstabulation:

In [35]:
indexList=['departamento','provincia']
aggregator={'case':['sum']}
worst_ProvYear_alarma_Frequency=worst_ProvYear_alarma.groupby(indexList,observed=True).agg(aggregator)
worst_ProvYear_alarma_Frequency

case
                                sum
departamento  provincia            
AMAZONAS      BAGUA               6
              UTCUBAMBA           6
ANCASH        CASMA               5
              SANTA               3
AREQUIPA      AREQUIPA            1
AYACUCHO      LA MAR              7
              SUCRE               1
CAJAMARCA     CAJAMARCA           1
              JAEN               10
CALLAO        CALLAO              2
CUSCO         LA CONVENCION       9
              QUISPICANCHI        1
HUANUCO       LEONCIO PRADO      10
              PUERTO INCA         2
ICA           CHINCHA             1
              ICA                 3
              NAZCA               1
              PALPA               2
JUNIN         CHANCHAMAYO         9
              SATIPO              4
LA LIBERTAD   ASCOPE              2
              CHEPEN              2
              TRUJILLO            6
LAMBAYEQUE    CHICLAYO            4
              FERREÑAFE           2
              LAMBAYEQUE          3
LIMA          LIMA                9
LORETO        ALTO AMAZONAS       1
              MAYNAS             13
MADRE DE DIOS TAMBOPATA          13
PASCO         OXAPAMPA           11
PIURA         HUANCABAMBA         1
              PIURA              10
              SULLANA             1
              TALARA              1
PUNO          CARABAYA            2
SAN MARTIN    HUALLAGA            1
              MARISCAL CACERES    2
              SAN MARTIN          8
              TOCACHE             2
TUMBES        TUMBES             12
              ZARUMILLA           1
UCAYALI       CORONEL PORTILLO   12

In [37]:
# flattening with counts
worst_ProvYear_alarma_Frequency.reset_index()

,departamento,provincia,case
,,,sum
0,AMAZONAS,BAGUA,6
1,AMAZONAS,UTCUBAMBA,6
2,ANCASH,CASMA,5
3,ANCASH,SANTA,3
4,AREQUIPA,AREQUIPA,1
5,AYACUCHO,LA MAR,7
6,AYACUCHO,SUCRE,1
7,CAJAMARCA,CAJAMARCA,1
8,CAJAMARCA,JAEN,10


In [38]:
# final look
worst_ProvYear_alarma_Frequency.columns=['case']
worst_ProvYear_alarma_Frequency.reset_index(inplace=True)
worst_ProvYear_alarma_Frequency

,departamento,provincia,case
0,AMAZONAS,BAGUA,6
1,AMAZONAS,UTCUBAMBA,6
2,ANCASH,CASMA,5
3,ANCASH,SANTA,3
4,AREQUIPA,AREQUIPA,1
5,AYACUCHO,LA MAR,7
6,AYACUCHO,SUCRE,1
7,CAJAMARCA,CAJAMARCA,1
8,CAJAMARCA,JAEN,10
9,CALLAO,CALLAO,2


Saving the frequencies in a file:

In [ ]:
worst_ProvYear_alarma_Frequency.to_csv('dataFiles/worst_ProvYear_alarma_Frequency.csv',index=False)

# The 'ALARM' symptoms level

## Aggregating

Let's check a previous data frame:

In [39]:
ByYearPlace_wide

,ano,departamento,provincia,case_sum_SIN_SEÑALES,case_sum_ALARMA,case_sum_GRAVE
0,2000,AMAZONAS,BAGUA,215.0,0.0,0.0
1,2000,AMAZONAS,UTCUBAMBA,58.0,0.0,0.0
2,2000,CAJAMARCA,CUTERVO,2.0,0.0,0.0
3,2000,CAJAMARCA,JAEN,16.0,0.0,0.0
4,2000,HUANUCO,LEONCIO PRADO,29.0,0.0,0.0
...,...,...,...,...,...,...
1305,2022,TUMBES,ZARUMILLA,89.0,5.0,0.0
1306,2022,UCAYALI,ATALAYA,542.0,92.0,2.0
1307,2022,UCAYALI,CORONEL PORTILLO,2680.0,499.0,23.0
1308,2022,UCAYALI,PADRE ABAD,412.0,87.0,2.0


This time, I want two variables:

In [40]:
indexList=['ano','departamento']
aggregator={'case_sum_SIN_SEÑALES':['sum'],'case_sum_ALARMA':['sum']}
ByYearPlace=ByYearPlace_wide.groupby(indexList,observed=True).agg(aggregator)
ByYearPlace.columns=['sum_SIN_SEÑALES','sum_ALARMA']
ByYearPlace.reset_index(inplace=True)
ByYearPlace

,ano,departamento,sum_SIN_SEÑALES,sum_ALARMA
0,2000,AMAZONAS,273.0,0.0
1,2000,CAJAMARCA,18.0,0.0
2,2000,HUANUCO,29.0,0.0
3,2000,JUNIN,7.0,0.0
4,2000,LA LIBERTAD,894.0,0.0
...,...,...,...,...
366,2022,PIURA,9296.0,1361.0
367,2022,PUNO,25.0,0.0
368,2022,SAN MARTIN,3229.0,907.0
369,2022,TUMBES,656.0,36.0


## Creating information

I will create a new variable:

In [41]:
ByYearPlace['rateAlarma']=(ByYearPlace['sum_ALARMA']/ByYearPlace['sum_SIN_SEÑALES'])
ByYearPlace['rateAlarma'].describe()

/Users/JoseManuel/opt/anaconda3/envs/SpatialDS_311_pandas2/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    371.000000
mean            inf
std             NaN
min        0.000000
25%        0.000000
50%        0.018216
75%        0.134195
max             inf
Name: rateAlarma, dtype: float64

We got _inf_ values:

In [42]:
import numpy as np #identify with numpy
ByYearPlace[np.isinf(ByYearPlace.rateAlarma)]

,ano,departamento,sum_SIN_SEÑALES,sum_ALARMA,rateAlarma
186,2013,AYACUCHO,0.0,1.0,inf


We need to make a decision. I did this:

In [43]:
ByYearPlace.loc[186,'rateAlarma']=1
ByYearPlace.drop(columns=['sum_SIN_SEÑALES','sum_ALARMA'],inplace=True)
ByYearPlace['rateAlarma'].describe()

count    371.000000
mean       0.111113
std        0.202358
min        0.000000
25%        0.000000
50%        0.018216
75%        0.134195
max        1.583333
Name: rateAlarma, dtype: float64

### Discretizing

Sometimes you need a numerical variable as an ordinal variable: 

In [44]:
edges=[-1, .1, .25, .5,.75,1,2]
theLabels=["less10%","10-25%","25-50","51-75%","75-100%","above100%"]
ByYearPlace["rateAlarma.cut"]=pd.cut(ByYearPlace['rateAlarma'], include_lowest=True,
                                     bins=edges, 
                                     labels=theLabels,
                                     ordered=True)

In [45]:
# we have
ByYearPlace

,ano,departamento,rateAlarma,rateAlarma.cut
0,2000,AMAZONAS,0.000000,less10%
1,2000,CAJAMARCA,0.000000,less10%
2,2000,HUANUCO,0.000000,less10%
3,2000,JUNIN,0.000000,less10%
4,2000,LA LIBERTAD,0.000000,less10%
...,...,...,...,...
366,2022,PIURA,0.146407,10-25%
367,2022,PUNO,0.000000,less10%
368,2022,SAN MARTIN,0.280892,25-50
369,2022,TUMBES,0.054878,less10%


We could check the yearly behavior:

In [46]:
ByYearPlace.groupby('ano').describe()

rateAlarma                                                              \
          count      mean       std       min       25%       50%       75%   
ano                                                                           
2000       11.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2001       14.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2002       15.0  0.000030  0.000116  0.000000  0.000000  0.000000  0.000000   
2003       12.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2004       13.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2005       16.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2006       14.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2007       15.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2008       14.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2009       15.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2010       14.0  0.008610  0.018810  0.000000  0.000000  0.000000  0.005710   
2011       15.0  0.099278  0.141586  0.000000  0.005208  0.043902  0.083672   
2012       16.0  0.150327  0.121813  0.000000  0.069813  0.122838  0.189647   
2013       18.0  0.375294  0.350709  0.000000  0.094267  0.274368  0.541558   
2014       16.0  0.285056  0.406160  0.000000  0.030391  0.097071  0.421336   
2015       18.0  0.314695  0.305296  0.000000  0.091302  0.275115  0.416486   
2016       18.0  0.190514  0.157750  0.007685  0.050252  0.158646  0.325561   
2017       19.0  0.151913  0.146412  0.000000  0.024195  0.087728  0.303617   
2018       19.0  0.142461  0.178510  0.000000  0.000000  0.059140  0.244079   
2019       19.0  0.127213  0.120950  0.000000  0.036147  0.083333  0.189534   
2020       19.0  0.169504  0.233548  0.005703  0.029017  0.093544  0.196144   
2021       21.0  0.135615  0.229660  0.000000  0.031120  0.072072  0.166667   
2022       20.0  0.135949  0.096298  0.000000  0.064062  0.127371  0.187478   

                
           max  
ano             
2000  0.000000  
2001  0.000000  
2002  0.000451  
2003  0.000000  
2004  0.000000  
2005  0.000000  
2006  0.000000  
2007  0.000000  
2008  0.000000  
2009  0.000000  
2010  0.069204  
2011  0.398804  
2012  0.487755  
2013  1.000000  
2014  1.583333  
2015  1.337931  
2016  0.509953  
2017  0.378788  
2018  0.519616  
2019  0.372638  
2020  1.000000  
2021  1.076923  
2022  0.381858

Let's do some **filtering**:

In [47]:
ByYearPlace=ByYearPlace[ByYearPlace.ano>=2012]
ByYearPlace.reset_index(drop=True,inplace=True)
ByYearPlace

,ano,departamento,rateAlarma,rateAlarma.cut
0,2012,AMAZONAS,0.126273,10-25%
1,2012,ANCASH,0.021111,less10%
2,2012,CAJAMARCA,0.177215,10-25%
3,2012,HUANUCO,0.309237,25-50
4,2012,JUNIN,0.061286,less10%
...,...,...,...,...
198,2022,PIURA,0.146407,10-25%
199,2022,PUNO,0.000000,less10%
200,2022,SAN MARTIN,0.280892,25-50
201,2022,TUMBES,0.054878,less10%


In [48]:
ByYearPlace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   ano             203 non-null    int64   
 1   departamento    203 non-null    object  
 2   rateAlarma      203 non-null    float64 
 3   rateAlarma.cut  203 non-null    category
dtypes: category(1), float64(1), int64(1), object(1)
memory usage: 5.3+ KB


In [49]:
# the category should be exported as pickle

ByYearPlace.to_pickle("dataFiles/ByYearPlace.pkl")

# World Fragility Data

## Concatenating


Let's visit this website: https://fundforpeace.org/what-we-do/country-risk-and-fragility-data/

There, you will find several excel files with the _Fragile States Index_ per year. Please, create folder **fragility** inside the folder **dataFiles**, where you will download the excel files from 2006 to 2023. 

In [ ]:
# Import libraries
import os
import glob
import pandas as pd

path = os.path.join('dataFiles','fragility','*.xlsx') # xlsx files in the folder
excel_files_names = glob.glob(path) #file names using pyhton's glob

# see the file names
excel_files_names


Let´s open each file (make sure you have previously installed **openpyxl**):

In [ ]:
allDFs=[] # all XLSX will be here!

import pandas as pd

for fileName in excel_files_names:
    currentFile=pd.read_excel(fileName)
    allDFs.append(currentFile)

In [ ]:
# amount of rows and columns:
for df,year in zip(allDFs,range(2006,2024)):
    print(df.shape,year)

In [ ]:
#dropping one year
allDFs_sub=allDFs[1::]

Putting all the dataframes column names into a list:

In [ ]:
allColumnNames=[] # I will write every column 
for df in allDFs_sub:
    allColumnNames.append(set(df.columns))# list of sets!

# this is what we have
allColumnNames

In [ ]:
# common columns
commonColumns=set.intersection(*allColumnNames) # expanding list of sets (*)
commonColumns

In [ ]:
commonColumns.symmetric_difference(set.union(*allColumnNames))

In [ ]:
allDFs_sameNames=[] # final DataFrame (with all DFs from 2013-2021
colnamesSorted=sorted(list(commonColumns)) # columns names sorted - must turn 'set' into 'list'

# making list of DFs
for df in allDFs_sub:
    allDFs_sameNames.append(df.loc[:,colnamesSorted]) 

# here it is
allDFs_sameNames

In [ ]:
# concatenating
allDFsConcat=pd.concat(allDFs_sameNames,ignore_index=True) # appending DFs using 'concat()'

#done!... see it:
allDFsConcat

In [ ]:
allDFsConcat.info()

In [ ]:
# value_counts can be used in object type
allDFsConcat.Year.value_counts()

In [ ]:
# keeping just the year value
yearAsNumber=[]
for y in allDFsConcat.Year:
    try:
        yearAsNumber.append(y.year)# the value from a date-time format
    except:
        yearAsNumber.append(y) # if not a datetime

#verifying
pd.Series(yearAsNumber).value_counts()

In [ ]:
# overwriting the year column
allDFsConcat['Year']=yearAsNumber

In [ ]:
# current order
allDFsConcat.columns.to_list()

In [ ]:
# this is a trick: setting columns as index
allDFsConcat.set_index(['Country','Year','Total'],inplace=True)
allDFsConcat.head()

Reordering columns:

In [ ]:
# dropping unneeded column
allDFsConcat.drop(columns='Rank',inplace=True)

In [ ]:
# indexes will be columns
allDFsConcat.reset_index(drop=False,inplace=True)

# see
allDFsConcat.head()

In [ ]:
# better ?
allDFsConcat.columns.to_list()

In [ ]:
# clean column names
allDFsConcat.columns=allDFsConcat.columns.str.replace(':\s',"_",regex=True)
allDFsConcat.columns=allDFsConcat.columns.str.replace('\s',"",regex=True)
#see
allDFsConcat.columns.to_list()

In [ ]:
# overwriting country
allDFsConcat['Country']=allDFsConcat.Country.str.upper()
allDFsConcat["Country"]=allDFsConcat.Country.str.strip()

## Reshaping after concatenation

We can find some problems that were created during the concatenation:

In [ ]:
# seeing long shape
fragileLong=allDFsConcat.iloc[:,:3]
fragileLong

In [ ]:
# to wide
fragileWide=pd.pivot_table(fragileLong,
               values='Total', # values to use
               index=['Country'], # unit of analysis
               columns=['Year']) # the values for NEW column
# see wide
fragileWide.head()

In [ ]:
# missing values in long format
fragileLong[fragileLong.isna().any(axis=1)]

In [ ]:
# what cells have missing values?
fragileWide[fragileWide.isna().any(axis=1)]

So, we got problems.

In [ ]:
# details
fragileWide[fragileWide.isna().any(axis=1)].index

In [ ]:
# prepare changes as dict:
changes={"CABO VERDE": "CAPE VERDE","CÔTE D'IVOIRE":"COTE D'IVOIRE", 
"CZECHIA":"CZECH REPUBLIC",
"SWAZILAND":"ESWATINI",
"ISRAEL AND WEST BANK":"PALESTINE",
"KYRGYZSTAN":"KYRGYZ REPUBLIC",
"NORTH MACEDONIA":"MACEDONIA",
"SLOVAKIA": "SLOVAK REPUBLIC"}

In [ ]:
# make changes using 'replace':
allDFsConcat.Country.replace(to_replace=changes,inplace=True)
# re create:
fragileLong=allDFsConcat.iloc[:,:3]

In [ ]:
# to wide shape again
fragileWide=pd.pivot_table(fragileLong,
               values='Total',
               index=['Country'],
               columns=['Year']).\
            reset_index(drop=False).\
            rename_axis(index=None, columns=None)

# verify missing
fragileWide[fragileWide.isna().any(axis=1)] 

In [ ]:
# new subset
allDFsConcat=allDFsConcat[allDFsConcat.Year>=2013]

In [ ]:
allDFsConcat[allDFsConcat.isna().any(axis=1)]

In [ ]:
allDFsConcat[allDFsConcat.Country=='PALESTINE']

In [ ]:
allDFsConcat[allDFsConcat.Country=='ISRAEL']

In [ ]:
allDFsConcat[allDFsConcat.Country=='ISRAEL'].to_dict(orient='records')[1]

In [ ]:
newYears=[]

for year in range(2013,2021):
    base=allDFsConcat[allDFsConcat.Country=='ISRAEL'].to_dict(orient='records')[1]
    base['Year']=year
    newYears.append(base)

pd.DataFrame(newYears)

In [ ]:

pd.concat([allDFsConcat,pd.DataFrame(newYears)], ignore_index = True)

In [ ]:
allDFsConcat=pd.concat([allDFsConcat,pd.DataFrame(newYears)], ignore_index = True)

In [ ]:
allDFsConcat.reset_index(drop=True, inplace=True)

In [ ]:
allDFsConcat

# Country Codes

## Merging

In [ ]:
# make sure to install 'html5lib', 'beautifulSoup4' and 'lxml'

codesLink='https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes'

allTablesWiki=pd.read_html(codesLink, flavor='bs4')

In [ ]:

allTablesWiki[0]

In [ ]:
# keep that on
countryCodes=allTablesWiki[0].copy()

In [ ]:
# check names
countryCodes.columns

In [ ]:
# keeping what is needed
countryCodes=countryCodes.iloc[:,[0,3,4]]

In [ ]:
countryCodes.columns

In [ ]:
# brute-force renaming
newNames=["Country","iso2","iso3"]
countryCodes.columns=newNames
countryCodes

In [ ]:
# bye symbols
countryCodes['Country']=countryCodes['Country'].str.normalize('NFKD').\
                        str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

In [ ]:
# check missing
countryCodes[countryCodes.isna().any(axis=1)]

In [ ]:
# easy fix
countryCodes.loc[countryCodes.Country=='NAMIBIA','iso2']="NA"

# something missing?
countryCodes[countryCodes.isna().any(axis=1)]

In [ ]:
# are these iso2 valid values?
[x for x in countryCodes.iso2 if len(x)>2]

In [ ]:
# wrong rows

badValues=[x for x in countryCodes.iso2 if len(x)>2]

countryCodes[countryCodes.iso2.isin(badValues)]

In [ ]:
# dropping wrong rows
countryCodes=countryCodes[~countryCodes.iso2.isin(badValues)] # filtering

countryCodes.reset_index(drop=True,inplace=True) # needed when rows are dropped

In [ ]:
#how many countries?
allDFsConcat.Country.unique().shape

In [ ]:
#how many countries?
countryCodes.Country.shape

In [ ]:
countryCodes[countryCodes.Country.str.startswith('UNITED')]

In [ ]:
countryCodes=countryCodes[countryCodes.iso3!='UMI']

In [ ]:
allDFsConcat[allDFsConcat.Country.str.startswith('UNITED')]

Let's use sets to determine the non coincidences:

In [ ]:
# only in countryCodes.Country NOT in allDFsConcat.Country
OnlyInCodes=set(countryCodes.Country)-set(allDFsConcat.Country)
OnlyInCodes

In [ ]:
# only in allDFsConcat.Country NOT in countryCodes.Country
OnlyInConcat=set(allDFsConcat.Country)-set(countryCodes.Country)
OnlyInConcat

## Fuzzy merging

We used the previous information to look for _possible_ matches (please install **thefuzz**):

In [ ]:
from thefuzz import process as fz

[(f,fz.extractOne(f, OnlyInCodes)) for f in sorted(OnlyInConcat)]

In [ ]:
# this may be clearer:

[(f,fz.extractOne(f, OnlyInCodes)) for f in sorted(OnlyInConcat)
 if fz.extractOne(f, OnlyInCodes)[1]>=90]

In [ ]:
# prepare a dict of changes

changesInCodes1={fz.extractOne(f, OnlyInCodes)[0]:f 
                 for f in sorted(OnlyInConcat)
                 if fz.extractOne(f, OnlyInCodes)[1] >=90}
#the dict
changesInCodes1

In [ ]:
countryCodes.Country.replace(to_replace=changesInCodes1,inplace=True)

In [ ]:
# second iteration

OnlyInCodes=set(countryCodes.Country)-set(allDFsConcat.Country)
OnlyInConcat=set(allDFsConcat.Country)-set(countryCodes.Country)

[(f,fz.extractOne(f, OnlyInCodes)) for f in sorted(OnlyInConcat)]

Based on last result, we may need manual changes:

In [ ]:
# see the strings in countryCodes:

countryCodes[countryCodes.Country.str.contains('LAO|KOREA|CZECH|CONGO',regex=True)]

In [ ]:
# second iteration (manual)

changesInCodes2={"KOREA (THE DEMOCRATIC PEOPLE'S REPUBLIC OF) [P]":'NORTH KOREA',
                 "KOREA (THE REPUBLIC OF) [Q]":"SOUTH KOREA",
                 "LAO PEOPLE'S DEMOCRATIC REPUBLIC (THE) [R]":"LAOS",
                 "CZECHIA [J]":'CZECH REPUBLIC',
                 "CONGO (THE) [H]":'CONGO REPUBLIC'}
countryCodes.Country.replace(to_replace=changesInCodes2,inplace=True)

Those changes now allow for a different result:

In [ ]:
OnlyInCodes=set(countryCodes.Country)-set(allDFsConcat.Country)
OnlyInConcat=set(allDFsConcat.Country)-set(countryCodes.Country)

[(f,fz.extractOne(f, OnlyInCodes)) for f in sorted(OnlyInConcat)]

In [ ]:
# we got it !
changesInCodes3={fz.extractOne(f, OnlyInCodes)[0]:f 
                 for f in sorted(OnlyInConcat)
                 if fz.extractOne(f, OnlyInCodes)[1] >=52}
#dict of matches
changesInCodes3

In [ ]:
# make the changes
countryCodes.Country.replace(to_replace=changesInCodes3,inplace=True)

In [ ]:
# confirming

OnlyInConcat=set(allDFsConcat.Country)-set(countryCodes.Country)
OnlyInConcat

When we recover the  most matches, we are ready to merge:

In [ ]:
fragilityCoded_2012_2023=allDFsConcat.merge(countryCodes,left_on='Country',right_on='Country') #merge on Country
fragilityCoded_2012_2023

# Transformation

In [ ]:
# we have these numeric columns
fragilityCoded_2012_2023.loc[:,'Total':'X1_ExternalIntervention'].columns

In [ ]:
# plotting
fragilityCoded_2012_2023.loc[:,'Total':'X1_ExternalIntervention'].boxplot()

In [ ]:
# all but 'Total'
fragilityCoded_2012_2023.loc[:,'C1_SecurityApparatus':'X1_ExternalIntervention'].boxplot()

In [ ]:
# Total variable by year
fragilityCoded_2012_2023[['iso3','Year','Total']].set_index('iso3').groupby('Year')['Total']

In [ ]:
# the result
fragilityCoded_2012_2023[['iso3','Year','Total']].set_index('iso3').groupby(['Year'])['Total'].transform(lambda x: 10*((x - x.min()) / (x.max()- x.min())))

In [ ]:
# another way
fragilityCoded_2012_2023[['iso3','Year','Total']].set_index('iso3').groupby(['Year'])['Total'].apply(lambda x: 10*(x-x.min())/(x.max()-x.min())).reset_index()

The two previous results differ in positions. Only the first one keeps the right index positions.

In [ ]:
# notice
fragilityCoded_2012_2023.iso3

In [ ]:
# saving MinMax result
total_mnmx=fragilityCoded_2012_2023[['iso3','Year','Total']].set_index('iso3').groupby(['Year'])['Total'].transform(lambda x: 10*((x - x.min()) / (x.max()- x.min())))
total_mnmx

In [ ]:
# as a list
total_mnmx.to_list()

In [ ]:
# adding new var
fragilityCoded_2012_2023.assign(Total_mnmx=total_mnmx.to_list())

In [ ]:
# actually adding
fragilityCoded_2012_2023=fragilityCoded_2012_2023.assign(Total_mnmx=total_mnmx.to_list())

In [ ]:
# the NA in NAmibia requires PKL:
fragilityCoded_2012_2023.to_pickle(os.path.join("dataFiles","fragility","fragilityCoded_2012_2023.pkl"))